# Barretenberg Notebook

This is a start on a Sage notebook for checking one's work. I think we shouldn't build this out systematically, but periodically update it as we find the need to build some new functionality.

# General Utilities

In [1]:
# Convert a 256-bit hex string. Outputs four 64-bit limbs that can be used to construct a native field element.
# Idea: s == barretenberg::fr(hex_to_field(s)).to_montgomery_form() is true as hex strings if s was in Montgomery form.
def hex_to_field(s):
    s = s[2:] # slice off leading 0x
    data = [s[48:64], s[32:48], s[16:32], s[0:16]]
    data = ["0x" + d for d in data]
    out = ', '.join(data)
    return "{" + out + "}"

hex_to_field('0x04410c360230a295b13d66d8d6c1a24c44311531e39c64f66c7301b49d85a46c')

'{0x6c7301b49d85a46c, 0x44311531e39c64f6, 0xb13d66d8d6c1a24c, 0x04410c360230a295}'

# Basic Structures

In [2]:
# The BN254 prime.
prime_r = 21888242871839275222246405745257275088548364400416034343698204186575808495617
Fr = GF(prime_r)
Polys.<X> = PolynomialRing(Fr)
print("Order of BN254 is: ", hex(prime_r))
print(Fr(-2))
print((X + 1) * (X - 1))

Order of BN254 is:  0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593f0000001
21888242871839275222246405745257275088548364400416034343698204186575808495615
X^2 + 21888242871839275222246405745257275088548364400416034343698204186575808495616


# Sumcheck Utilities
Here we generate generate a two-gate example of a sumcheck calculation. The output of `get_values` is the expected `Univariate` produced after the first round.

In [3]:
def get_values(poly):
    return [poly(i) for i in range(4)]

In [4]:
def convert_to_polys(edge_list):
    return [edge[0] * (1-X) + edge[1] * X for edge in edge_list]

In [5]:
edge_list = [[0, 1], [0, 1], [0, 2], [0, 0], [1, 1], [0, 1], [0, -1], [0, 0]]
w_l, w_r, w_o, q_m, q_l, q_r, q_o, q_c = convert_to_polys(edge_list)
[hex(val) for val in get_values(q_m * w_l * w_r + q_l * w_l + q_r * w_r + q_o * w_o + q_c)]

['0x0',
 '0x0',
 '0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593efffffff',
 '0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593effffffb']

In [6]:
# edge_list = [[0,0] for _ in range(8)]
# w_l, w_r, w_o, q_m, q_l, q_r, q_o, q_c = convert_to_polys(edge_list)
# get_values(q_m * w_l * w_r + q_l * w_l + q_r * w_r + q_o * w_o + q_c)

In [7]:
u_2 = 0x2cb413b29041834eb4c859d50a771208a5e3dec2cd9bb2a1a13e5a57c6160691

In [8]:
folded = [[poly(u_2), 0] for poly in [w_l, w_r, w_o, q_m, q_l, q_r, q_o, q_c]]
print(folded)

[[20219933756566846493536316042331920548925056837893251409966172577112016684689, 0], [20219933756566846493536316042331920548925056837893251409966172577112016684689, 0], [18551624641294417764826226339406566009301749275370468476234140967648224873761, 0], [0, 0], [1, 0], [20219933756566846493536316042331920548925056837893251409966172577112016684689, 0], [1668309115272428728710089702925354539623307562522782933732031609463791810928, 0], [0, 0]]


In [9]:
folded == [[u_2, 0], [u_2, 0], [2*u_2, 0], [0, 0], [1, 0], [u_2, 0], [-u_2, 0], [0, 0]]

True

In [10]:
[hex(Fr(first)) for [first, second] in folded]

['0x2cb413b29041834eb4c859d50a771208a5e3dec2cd9bb2a1a13e5a57c6160691',
 '0x2cb413b29041834eb4c859d50a771208a5e3dec2cd9bb2a1a13e5a57c6160691',
 '0x2903d8f23f516673b1406df3936ccbb42393d53d217df4b1fe9abf1b9c2c0d21',
 '0x0',
 '0x1',
 '0x2cb413b29041834eb4c859d50a771208a5e3dec2cd9bb2a1a13e5a57c6160691',
 '0x3b03ac050f01cdb0387ebe1770a465482500985ac1dbdefa2a39b3c29e9f970',
 '0x0']

In [11]:
w_l, w_r, w_o, q_m, q_l, q_r, q_o, q_c = convert_to_polys(folded)
[hex(val) for val in get_values(q_m * w_l * w_r + q_l * w_l + q_r * w_r + q_o * w_o + q_c)]

['0x18ea803098b5a8b9d02047be7a4f24cf4db4c25696323b82b34625f98827c179',
 '0x0',
 '0x18ea803098b5a8b9d02047be7a4f24cf4db4c25696323b82b34625f98827c179',
 '0x2e163dca0736293cfe0938ce639e282e66b38c965560ce84554acbe409f05e2']